### Forecasting Models on Овцы и козы 

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

from pylab import rcParams
from IPython.display import display
import math
from prophet import Prophet
pd.set_option('display.max_columns', 130)


import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter("ignore", category=InterpolationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)



#### Data preparation

In [8]:
df = pd.read_excel("../../Data cleansing/output data/Просуммированные по категориям с доп регрессорами.xlsx")
df.head(5)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Верблюды,АКМОЛИНСКАЯ ОБЛАСТЬ,0.00,0.00,0.40,0.00,0.00,0.00,0.09,1.00,0.00,0.00,0.20,0.00,0.00,0.18,0.28,0.00,0.00,0.40,0.00,0.00,0.65,0.00,0.31,1.00,0.00,0.00,0.00,0.00,0.00,2.01,0.00,1.20,0.00,0.00,2.18,0.39,0.00,0.00,1.04,0.00,0.14,2.08,0.00,0.00,0.00,0.00,0.00,0.30,0.00,0.00,0.00,0.66,0.00,0.33,0.00,0.9,0.00,0.00,0.00,10.08,0.00,0.0,0.00,0.0,0.00,0.54,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.36,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00
1,Верблюды,АКТЮБИНСКАЯ ОБЛАСТЬ,101.98,67.47,374.84,115.59,218.72,14.15,19.77,3.00,39.16,46.16,238.56,463.35,109.04,72.94,384.96,114.35,221.89,10.93,18.79,3.50,38.53,46.64,216.08,472.07,110.32,68.17,380.15,127.12,217.61,14.98,21.26,7.14,44.63,51.78,239.43,513.33,115.70,69.40,371.53,130.17,218.41,14.59,21.76,7.19,45.97,55.48,252.68,527.66,117.94,70.37,385.75,117.80,218.45,15.54,21.78,7.2,47.06,58.02,257.54,543.35,119.80,71.3,396.90,121.8,228.10,15.90,21.9,7.2,63.00,59.90,260.20,552.10,121.60,71.8,398.40,128.50,228.80,13.10,22.50,7.20,63.90,61.90,150.70,554.90,121.30,73.3,404.20,128.70,234.90,13.60,23.60,6.3,49.80,62.4,153.70,552.8,118.07,69.17,384.21,121.01,223.46,11.76,21.39,5.52,48.18,61.39,150.87,531.79,119.50,72.60,391.0,126.1,229.10,12.2,24.10,5.7,49.87,63.00,154.3,538.40
2,Верблюды,АЛМАТИНСКАЯ ОБЛАСТЬ,1.00,0.20,51.60,25.40,0.00,61.90,76.87,95.57,16.09,0.10,10.80,46.78,15.06,13.70,131.80,9.20,17.89,130.12,2.00,0.00,4.99,2.06,4.90,44.48,15.06,23.94,41.60,19.54,3.00,15.14,18.50,14.82,14.52,12.60,9.92,43.11,20.72,2.96,15.24,0.00,1.00,20.03,3.44,7.77,117.25,1.00,0.00,20.34,21.95,3.04,17.91,26.81,9.16,18.80,13.37,0.0,17.29,15.90,4.69,23.10,10.85,9.7,42.75,0.0,6.21,21.10,4.0,2.6,28.10,4.53,79.50,93.18,23.70,11.2,16.00,2.00,3.10,19.62,2.30,4.85,11.18,2.80,9.65,36.44,10.60,11.5,26.10,7.20,12.75,11.20,11.20,27.0,25.26,17.9,21.90,12.5,16.55,16.81,22.78,12.10,12.44,6.85,39.65,17.22,8.85,27.48,0.48,29.72,18.90,17.40,12.5,16.4,11.60,16.7,15.70,24.0,7.70,6.90,5.4,12.60
3,Верблюды,АТЫРАУСКАЯ ОБЛАСТЬ,213.89,167.70,306.60,164.97,342.57,192.00,43.60,113.03,262.37,193.97,308.17,1087.33,325.10,190.60,301.10,154.84,328.50,220.30,66.10,118.20,234.90,196.51,270.92,974.18,303.30,167.41,323.54,182.34,349.10,249.40,57.57,88.40,221.21,199.63,278.80,964.02,305.22,159.62,326.87,159.50,367.20,257.50,60.01,78.10,253.27,332.00,355.05,989.86,280.08,165.05,334.51,134.85,367.97,321.22,126.22,93.7,263.10,366.66,348.54,1041.38,293.40,154.8,339.44,143.6,405.80,308.35,88.5,130.5,575.30,458.70,357.62,998.10,292.76,188.3,487.02,129.28,421.44,351.20,95.77,97.54,624.80,631.65,499.30,1179.66,286.67,189.8,513.16,171.46,402.10,432.22,121.53,126.2,637.50,651.7,482.94,1412.2,307.85,200.59,466.60,182.63,414.56,405.19,184.80,154.96,717.92,719.01,522.83,1164.77,323.81,423.76,441.9,219.1,487.69,446.9,191.43,133.5,730.34,709.68,537.6,930.28
4,Верблюды,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.30,0.00,0.00,1.14,6.12,4.70

In [9]:
df['Показатель'].unique()

array(['Верблюды', 'КРС', 'Лошади', 'Молоко', 'Овцы и козы', 'Птица',
       'Свиньи', 'Яйца', 'Температура', 'Поголовье: КРС',
       'Поголовье: лошади', 'Поголовье: овцы и козы', 'Поголовье: свиньи',
       'Поголовье: птица домашняя', 'Поголовье: верблюды', 'Осадки',
       'Цена: Говядина', 'Цена: Баранина', 'Цена: Молоко', 'Цена: Яйца'],
      dtype=object)

In [13]:
df_sheep = df[df['Показатель'].isin(['Овцы и козы', 'Температура', 'Осадки', 'Поголовье: овцы и козы', 'Цена: Баранина'])]
df_sheep.sample(10)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
240,Поголовье: овцы и козы,ПАВЛОДАРСКАЯ ОБЛАСТЬ,551585.000000,592496.000000,672595.000000,724288.000000,728825.000000,710325.000000,654221.000000,630864.000000,605560.000000,580781.000000,560716.000000,516262.000000,574139.000000,602818.000000,684620.000000,743240.000000,740901.000000,722870.000000,651406.000000,626877.000000,605671.000000,582693.000000,562400.000000,547197.000000,576902.000000,625574.000000,707206.000000,755103.000000,758070.000000,745962.000000,662972.000000,632112.000000,606298.000000,583931.000000,560775.000000,526833.000000,542046.000000,583184.000000,658247.000000,700129.000000,708783.000000,700837.000000,637238.000000,606722.000000,582486.000000,561893.000000,537517.000000,536881.000000,553662.000000,608943.000000,677260.000000,717113.000000,723411.000000,713400.000000,644515.000000,622937.000000,597385.000000,576615.000000,551167.000000,551586.000000,489598.000000,503559.000000,553398.000000,620392.000000,648886.000000,655167.000000,646126.000000,590970.000000,571341.000000,546065.000000,526952.000000,504633.000000,565484.000000,588606.000000,634547.000000,704769.000000,743007.000000,752238.000000,740717.000000,677994.000000,655832.000000,630886.000000,610669.000000,584517.000000,617804.000000,643070.000000,691597.000000,769438.000000,812533.000000,827640.000000,818918.000000,723095.000000,700243.000000,674981.000000,654533.000000,629278.000000,23785.000000,24972.000000,25396.000000,25685.000000,28462.000000,32191.000000,31832.000000,30989.000000,30113.000000,35852.000000,36100.000000,36193.000000,506802.000000,536181.000000,580169.000000,647971.000000,697025.000000,724061.000000,715267.000000,691922.000000,672373.000000,649128.000000,620617.000000,599308.000000
180,Температура,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,-10.364516,-9.839286,-4.858065,6.830000,14.774194,20.636667,21.306452,19.096774,11.086667,7.603226,-3.776667,-2.932258,-16.341935,-8.731034,-0.761290,9.346667,12.841935,19.783333,21.300000,17.729032,15.393333,1.293548,-10.358621,-5.954839,-9.725806,-10.403571,-7.229032,8.430000,15.470968,21.180000,20.858065,18.322581,12.046667,5.222581,-2.480000,-11.809677,-20.635484,-13.342857,-1.751613,7.150000,11.012903,20.706667,19.845161,19.122581,11.233333,6.422581,-5.160000,-16.754839,-12.674194,-12.675000,-0.809677,8.063333,11.645161,17.716667,20.687097,20.290323,13.580000,6.916129,-6.246667,-6.516129,-8.258065,-6.093103,-2.516129,11.353333,16.983871,18.890000,21.506452,19.683871,12.106667,5.516129,-4.503333,-13.696774,-17.258065,-8.114286,-3.151613,6.803333,15.825806,17.893333,21.835484,19.100000,12.316667,4.316129,-3.126667,-7.054839,-10.690323,-10.889286,-3.716129,8.886667,18.025806,20.323333,21.216129,17.593548,14.573333,6.277419,-4.793333,-15.751613,-10.603226,-8.378571,0.332258,5.336667,13.312903,21.146667,22.722581,20.293548,13.590000,8.777419,0.870000,-10.416129,-9.229032,-14.289655,-1.012903,9.290000,15.641935,20.903333,22.509677,21.361290,10.743333,7.209677,-1.160000,-8.322581
177,Температура,МАНГИСТАУСКАЯ ОБЛАСТЬ,-0.445161,1.057143,5.83

In [14]:
# Step 1: Pivot to wide format (each indicator becomes columns of periods)
df_wide = df_sheep.pivot(index="Регион", columns="Показатель")

# Step 2: Flatten multi-level columns: ('2015-01', 'КРС') → 'КРС_2015-01'
df_wide.columns = [f"{col[1]}_{col[0]}" for col in df_wide.columns]
df_wide = df_wide.reset_index()

# Step 3: Melt: one row per region-period-indicator
df_melted = df_wide.melt(id_vars="Регион", var_name="indicator_period", value_name="value")

# Step 4: Extract 'Период' and 'Показатель' from the combined column
df_melted["Период"] = df_melted["indicator_period"].str.extract(r"_(\d{4}-\d{2})$")
df_melted["Показатель"] = df_melted["indicator_period"].str.extract(r"^(.+)_\d{4}-\d{2}")

# Step 5: Pivot again to get final modeling format: one row per region+period, one column per indicator
df_sheep = df_melted.pivot_table(index=["Регион", "Период"], columns="Показатель", values="value").reset_index()
print(df_sheep.groupby("Регион").size().reset_index(name="Количество строк"))
df_sheep

                            Регион  Количество строк
0              АКМОЛИНСКАЯ ОБЛАСТЬ               120
1              АКТЮБИНСКАЯ ОБЛАСТЬ               120
2              АЛМАТИНСКАЯ ОБЛАСТЬ               120
3               АТЫРАУСКАЯ ОБЛАСТЬ               120
4   ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
5                          ГАЛМАТЫ               120
6                          ГАСТАНА               120
7                         ГШЫМКЕНТ                79
8               ЖАМБЫЛСКАЯ ОБЛАСТЬ               120
9    ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
10          КАРАГАНДИНСКАЯ ОБЛАСТЬ               120
11            КОСТАНАЙСКАЯ ОБЛАСТЬ               120
12          КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ               120
13           МАНГИСТАУСКАЯ ОБЛАСТЬ               120
14                    ОБЛАСТЬ АБАЙ                31
15                  ОБЛАСТЬ ЖЕТІСУ                31
16                  ОБЛАСТЬ ҰЛЫТАУ                31
17            ПАВЛОДАРСКАЯ ОБЛАСТЬ            

Показатель,Регион,Период,Овцы и козы,Осадки,Поголовье: овцы и козы,Температура,Цена: Баранина
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-01,602.28,9.8,470582.0,-12.490323,100.000000
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-02,636.18,9.8,524810.0,-10.192857,100.000000
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-03,740.73,8.3,562860.0,-5.870968,100.000000
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-04,690.81,8.8,599518.0,4.490000,99.200000
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-05,869.73,42.8,632220.0,14.574194,99.200000
...,...,...,...,...,...,...,...
2166,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,3378.76,0.0,5034130.0,27.874194,179.022742
2167,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,4625.15,0.5,4967650.0,20.766667,179.201765
2168,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-10,4443.92,13.6,4881045.0,13.200000,179.380967
2169,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-11,5402.06,12.1,4779726.0,6.500000,179.380967


In [15]:
df_sheep.to_excel("Датасет по овцы и козы.xlsx", index=False)

### Multivariate Regression

#### check statistical significance

In [ ]:
# Container for collecting regression results
results_list = []

# Loop over each region
for region in df_eggs["Регион"].unique():
    df_region = df_eggs[df_eggs["Регион"] == region][["КРС", "Поголовье: КРС", "Температура", "Осадки"]].dropna()

    if df_region.shape[0] < 10:
        print(f"⚠️ Skipping {region} — insufficient data ({len(df_region)} rows)")
        continue

    y = df_region["КРС"]
    X = df_region[["Поголовье: КРС", "Температура", "Осадки"]]
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()

    # ✅ Print model summary
    print(f"\n📍 Регион: {region}")
    print(model.summary())

    # ✅ Collect results
    for var in model.params.index:
        results_list.append({
            "Регион": region,
            "Переменная": var,
            "Коэффициент": model.params[var],
            "p-value": model.pvalues[var],
            "R-квадрат": model.rsquared
        })

# Convert to DataFrame
results_df = pd.DataFrame(results_list)

# Export to Excel
# results_df.to_excel("krs_linear_regression_results.xlsx", index=False)

print("\n Готово! Результаты экспортированы в 'krs_linear_regression_results.xlsx'")


#### forecast using MLR

In [ ]:
# Forecasting horizon
forecast_horizon = 3

# Container for summary results
results = []

# Loop through each region
for region in df_eggs["Регион"].unique():
    if region == "РЕСПУБЛИКА КАЗАХСТАН":
        continue  # ⛔ skip this region
    df_r = df_eggs[df_eggs["Регион"] == region].sort_values("Период")

    if df_r.shape[0] < forecast_horizon + 12:
        continue  # skip if not enough data

    # Train-test split
    df_train = df_r.iloc[:-forecast_horizon]
    df_test = df_r.iloc[-forecast_horizon:]

    X_train = df_train[["Поголовье: КРС", "Температура", "Осадки"]]
    y_train = df_train["КРС"]
    X_test = df_test[["Поголовье: КРС", "Температура", "Осадки"]]
    y_test = df_test["КРС"]

    # Fit model
    model = LinearRegression().fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    mape = (abs((y_test - y_pred) / y_test).mean()) * 100

    results.append({
        "Регион": region,
        "RMSE": rmse,
        "MAE": mae,
        "MAPE (%)": mape,
        "R² (Train)": model.score(X_train, y_train)
    })

# Save results
results_df = pd.DataFrame(results)
# results_df.to_excel("mlr_forecast_evaluation_by_region.xlsx", index=False)

results_df


In [ ]:
# Calculate and print mean metrics
mean_mape = results_df["MAPE (%)"].mean()
mean_rmse = results_df["RMSE"].mean()
mean_mae = results_df["MAE"].mean()

print(f"Средние метрики по регионам:")
print(f"Средний MAPE: {mean_mape:.2f}%")
print(f"Средний RMSE: {mean_rmse:.2f}")
print(f"Средний MAE: {mean_mae:.2f}")

#### regression for Kazakhstan - Поголовье not statistically significant

In [ ]:
df_kazakhstan = df_eggs[df_eggs['Регион'] == 'РЕСПУБЛИКА КАЗАХСТАН'][["КРС", "Поголовье: КРС"]].dropna()
y = df_kazakhstan["КРС"]
X = df_kazakhstan[["Поголовье: КРС"]]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

# Print model summary
print("Регион: РЕСПУБЛИКА КАЗАХСТАН")
print(model.summary())

In [ ]:
# from sklearn.linear_model import Ridge

# # Forecasting horizon
# forecast_horizon = 3

# # Container for summary results
# results = []

# # Loop through each region
# for region in df_model["Регион"].unique():
#     if region == "РЕСПУБЛИКА КАЗАХСТАН":
#         continue  # ⛔ skip this region
#     df_r = df_model[df_model["Регион"] == region].sort_values("Период")

#     if df_r.shape[0] < forecast_horizon + 12:
#         continue  # skip if not enough data

#     # Train-test split
#     df_train = df_r.iloc[:-forecast_horizon]
#     df_test = df_r.iloc[-forecast_horizon:]

#     X_train = df_train[["Поголовье: КРС", "Температура", "Осадки"]]
#     y_train = df_train["КРС"]
#     X_test = df_test[["Поголовье: КРС", "Температура", "Осадки"]]
#     y_test = df_test["КРС"]

#     # Fit model
#     model = Ridge(alpha=1.0).fit(X_train, y_train)
#     y_pred = model.predict(X_test)

#     # Calculate metrics
#     rmse = np.sqrt(mean_squared_error(y_test, y_pred))
#     mae = mean_absolute_error(y_test, y_pred)
#     mape = (abs((y_test - y_pred) / y_test).mean()) * 100

#     results.append({
#         "Регион": region,
#         "RMSE": rmse,
#         "MAE": mae,
#         "MAPE (%)": mape,
#         "R² (Train)": model.score(X_train, y_train)
#     })

#     # Plot actual vs forecast
#     plt.figure(figsize=(6, 3))
#     plt.plot(df_test["Период"], y_test, marker="o", label="Факт")
#     plt.plot(df_test["Период"], y_pred, marker="s", label="Прогноз")
#     plt.title(f"{region} — Прогноз объема КРС на {forecast_horizon} месяца")
#     plt.xlabel("Месяц")
#     plt.ylabel("Объем КРС")
#     plt.legend()
#     plt.xticks(rotation=45)
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()

# # Save results
# results_df = pd.DataFrame(results)
# results_df.to_excel("ridge_forecast_evaluation_by_region.xlsx", index=False)

# print("✅ Готово! Метрики и графики прогноза рассчитаны.")


In [ ]:
# # Forecasting horizon
# forecast_horizon = 12

# # Container for summary results
# results = []

# # Loop through each region
# for region in df_model["Регион"].unique():
#     if region == "РЕСПУБЛИКА КАЗАХСТАН":
#         continue  # ⛔ skip this region
#     df_r = df_model[df_model["Регион"] == region].sort_values("Период")

#     if df_r.shape[0] < forecast_horizon + 12:
#         continue  # skip if not enough data

#     # Train-test split
#     df_train = df_r.iloc[:-forecast_horizon]
#     df_test = df_r.iloc[-forecast_horizon:]

#     X_train = df_train[["Поголовье: КРС", "Температура", "Осадки"]]
#     y_train = df_train["КРС"]
#     X_test = df_test[["Поголовье: КРС", "Температура", "Осадки"]]
#     y_test = df_test["КРС"]

#     # Fit model
#     model = LinearRegression().fit(X_train, y_train)
#     y_pred = model.predict(X_test)

#     # Calculate metrics
#     rmse = np.sqrt(mean_squared_error(y_test, y_pred))
#     mae = mean_absolute_error(y_test, y_pred)
#     mape = (abs((y_test - y_pred) / y_test).mean()) * 100

#     results.append({
#         "Регион": region,
#         "RMSE": rmse,
#         "MAE": mae,
#         "MAPE (%)": mape,
#         "R² (Train)": model.score(X_train, y_train)
#     })

#     # 🟦 Optional: Plot actual vs forecast
#     plt.figure(figsize=(8, 4))
#     plt.plot(df_test["Период"], y_test, marker="o", label="Факт")
#     plt.plot(df_test["Период"], y_pred, marker="s", label="Прогноз")
#     plt.title(f"{region} — Прогноз объема КРС на {forecast_horizon} месяца")
#     plt.xlabel("Месяц")
#     plt.ylabel("Объем КРС")
#     plt.legend()
#     plt.xticks(rotation=45)
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()

# # Save results
# results_df = pd.DataFrame(results)
# results_df.to_excel("mlr_forecast_evaluation_by_region_12m.xlsx", index=False)

# print("✅ Готово! Метрики и графики прогноза рассчитаны.")


In [ ]:
# def sliding_window_evaluation(df_r, forecast_horizon=3, window_count=9):
#     metrics = []

#     # Ensure sorted by time
#     df_r = df_r.sort_values("Период").reset_index(drop=True)

#     for i in range(window_count):
#         # Define train and test split
#         train_end = i + 12  # require 12+ months for training
#         test_start = train_end
#         test_end = test_start + forecast_horizon

#         if test_end > len(df_r):
#             break

#         df_train = df_r.iloc[:train_end]
#         df_test = df_r.iloc[test_start:test_end]

#         X_train = df_train[["Поголовье: КРС", "Температура", "Осадки"]]
#         y_train = df_train["КРС"]
#         X_test = df_test[["Поголовье: КРС", "Температура", "Осадки"]]
#         y_test = df_test["КРС"]

#         # Fit model
#         model = LinearRegression().fit(X_train, y_train)
#         y_pred = model.predict(X_test)

#         # Calculate metrics
#         rmse = np.sqrt(mean_squared_error(y_test, y_pred))
#         mae = mean_absolute_error(y_test, y_pred)
#         mape = (np.abs((y_test - y_pred) / y_test).mean()) * 100

#         metrics.append({"RMSE": rmse, "MAE": mae, "MAPE": mape})

#     # Aggregate metrics over all sliding windows
#     return pd.DataFrame(metrics).mean().to_dict()

# results = []

# for region in df_model["Регион"].unique():
#     if region == "РЕСПУБЛИКА КАЗАХСТАН":
#         continue  # Optional: exclude a region

#     df_r = df_model[df_model["Регион"] == region][["Период", "КРС", "Поголовье: КРС", "Температура", "Осадки"]].dropna()

#     if df_r.shape[0] < 24:
#         continue  # need enough data for multiple windows

#     metrics = sliding_window_evaluation(df_r)

#     results.append({
#         "Регион": region,
#         "Avg RMSE": metrics["RMSE"],
#         "Avg MAE": metrics["MAE"],
#         "Avg MAPE (%)": metrics["MAPE"]
#     })

# # Save result
# results_df = pd.DataFrame(results)
# results_df.to_excel("sliding_window_forecast_evaluation.xlsx", index=False)

# print("✅ Готово! Средние метрики по скользящим окнам сохранены.")



### Autoregressive Integrated Moving Average (ARIMA)

In [ ]:
# Container for forecast evaluation results
results = []

# Loop over each region
for region in df_eggs["Регион"].unique():
    df_r = df_eggs[df_eggs["Регион"] == region].sort_values("Период")
    y = df_r["КРС"].reset_index(drop=True)

    if y.isna().sum() > 0 or len(y) < 24:  # Need at least 2 full seasons
        continue

    # Split data
    test_size = 3
    y_train = y[:-test_size]
    y_test = y[-test_size:]

    try:
        # Auto SARIMA model selection
        model = auto_arima(
            y_train,
            seasonal=True,        # Enable SARIMA
            m=12,                 # Season length (12 months)
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )

        # Forecast
        forecast = model.predict(n_periods=test_size)

        # Evaluation
        rmse = np.sqrt(mean_squared_error(y_test, forecast))
        mae = mean_absolute_error(y_test, forecast)
        mape = (np.abs((y_test - forecast) / y_test).mean()) * 100

        results.append({
            "Регион": region,
            "SARIMA order": model.order,
            "Seasonal order": model.seasonal_order,
            "RMSE": rmse,
            "MAE": mae,
            "MAPE (%)": mape
        })

        # Plot actual vs forecast
        plt.figure(figsize=(8, 4))
        plt.plot(y.index, y, color="lightgray", label="Все данные")
        plt.plot(y_test.index, y_test, marker="o", color="black", label="Факт (тест)")
        plt.plot(y_test.index, forecast, marker="s", linestyle="--", color="red", label="Прогноз")
        plt.axvline(len(y_train) - 1, color="gray", linestyle="--", label="Граница Train/Test")
        plt.title(f"{region} — SARIMA{model.order}x{model.seasonal_order} Прогноз КРС")
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"⚠️ Ошибка в {region}: {e}")

# Save results to Excel
results_df = pd.DataFrame(results)
results_df
# results_df.to_excel("sarima_forecast_evaluation_all_regions.xlsx", index=False)


In [ ]:
# Calculate and print mean metrics
mean_mape = results_df["MAPE (%)"].mean()
mean_rmse = results_df["RMSE"].mean()
mean_mae = results_df["MAE"].mean()

print(f"Средние метрики по регионам:")
print(f"Средний MAPE: {mean_mape:.2f}%")
print(f"Средний RMSE: {mean_rmse:.2f}")
print(f"Средний MAE: {mean_mae:.2f}")